In [ ]:
from doc_collection.core import *

In [ ]:
# hide
import warnings
warnings.filterwarnings("ignore")

# doc_collection

> A tool for collecting documentation of python libs.

## Install


`git clone https://github.com/omemaxim/doc_collection`

## How to use

Firstly, you need to pip list of packages to be able to collect data from it:

In [ ]:
pip_top_hundred()

Now you are ready to extract data from all libs you have in your Python (where are can be some exceptions):

In [ ]:
extract()

--------------- exception during tensorflow-io-gcs-filesystem documentation extracting
--------------- exception during theano documentation extracting


,text,name,library
0,pandas.DataFrame.columns. AxisProperty\n\n ...,pandas.DataFrame.columns,pandas
1,pandas.Series.index. AxisProperty\n\n The i...,pandas.Series.index,pandas
2,pandas.DataFrame.index. AxisProperty\n\n Th...,pandas.DataFrame.index,pandas
3,ipykernel.comm.Comm.topic. Bytes in module tra...,ipykernel.comm.Comm.topic,ipykernel
4,ipywidgets.Audio.value. Bytes in module traitl...,ipywidgets.Audio.value,ipykernel
...,...,...,...
56399,aiohttp.ClientResponse.url. reify\n\n,aiohttp.ClientResponse.url,aiohttp
56400,aiohttp.ClientResponse.url_obj. reify\n\n,aiohttp.ClientResponse.url_obj,aiohttp
56401,aiohttp.ClientResponse.history. reify\n\n A...,aiohttp.ClientResponse.history,aiohttp
56402,aiohttp.BodyPartReader.filename. reify\n\n ...,aiohttp.BodyPartReader.filename,aiohttp


Command above will return DataFrame of three columns: __text__ contains documentation of an object, __name__ contains the name and __library__ contains library of an object.